In [ ]:
#made a copy of the top pick file to turn into something more than a top 10 picker based off the most recent monthly returns

In [1]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import os

In [2]:
# Load the portfolio data
#data_dir = os.path.join('..', 'Data')
#input_path = os.path.join('Data', 'portfolio_pick.csv')

#xialin told me to do it @xialin pls don't get mad at me lol
input_path = r'C:\Users\bsung\OneDrive\Documents\GitHub\PortfolioProject\Data\portfolio_pick.csv'

monthly_returns = pd.read_csv(input_path)

In [3]:
def get_last_market_date():
        today = datetime.today()
        if today.weekday() == 5:  # Saturday
            return today - timedelta(days=1)
        elif today.weekday() == 6:  # Sunday
            return today - timedelta(days=2)
        return today
    
last_market_date = get_last_market_date().strftime('%Y-%m-%d')
    
    # Fetch stock prices
def get_price(ticker):
    try:
        stock = yf.Ticker(ticker)
        history = stock.history(period='5d')  # Fetch last 5 days to ensure data availability
        if not history.empty:
            return history['Close'].dropna().iloc[-1]  # Get the most recent available closing price
    except Exception as e:
        print(f"Error fetching price for {ticker}: {e}")
    return None
    
monthly_returns['Price'] = monthly_returns['Stock'].apply(get_price)

In [4]:

def calculate_most_recent_monthly_return(ticker):
    stock = yf.Ticker(ticker)
    hist = stock.history(period="3mo")
    hist['Monthly Return'] = hist['Close'].pct_change(periods=21)
    hist = hist[['Monthly Return']].dropna()

    today = datetime.today()
    most_recent_return = hist.iloc[-1] if today.day == 1 else hist.iloc[-2]

    return most_recent_return['Monthly Return']

# Calculate monthly returns for each stock
stock_dataframes = {ticker: calculate_most_recent_monthly_return(ticker) for ticker in monthly_returns['Stock']}

# Create a DataFrame with the results
result_df = pd.DataFrame(list(stock_dataframes.items()), columns=['Stock', 'Monthly Return'])

# Get the top 10 tickers with the highest returns
top_10_df = result_df.nlargest(10, 'Monthly Return')



In [5]:
# Output the result to a CSV file
results_dir = os.path.join('..', 'results')

os.makedirs(results_dir, exist_ok=True)

file_path = os.path.join(results_dir, 'top10portfoliopick.csv')

top_10_df.to_csv(file_path, index=False)

print(f"saved to {file_path}")

saved to ..\results\top10portfoliopick.csv
